In [ ]:
"""
导入必要的库
"""
import sys
import os
sys.path.append('..')  # 添加项目根目录到路径

from src.game.chess_board import ChessBoard
from src.game.chess_game import ChessGame
from src.neural_network.model import AlphaZeroNet
from src.mcts.mcts import MCTS
from config.config import config

import chess
import chess.svg
from IPython.display import SVG, display, clear_output
import time


In [ ]:
"""
创建交互式游戏可视化器类
"""
class InteractiveGameVisualizer:
    def __init__(self, model_path=None):
        """初始化可视化器"""
        self.board = chess.Board()
        
        # 加载模型（如果提供）
        if model_path and os.path.exists(model_path):
            self.model = AlphaZeroNet(config)
            self.model.load_state_dict(torch.load(model_path))
            self.model.eval()
            self.mcts = MCTS(self.model, config)
        else:
            self.model = None
            self.mcts = None
    
    def display_board(self):
        """显示当前棋盘状态"""
        clear_output(wait=True)
        display(SVG(chess.svg.board(self.board, size=400)))
        
        # 显示额外信息
        if self.board.is_game_over():
            print("游戏结束!")
            print("结果:", self.board.outcome().result())
        else:
            print("轮到", "白方" if self.board.turn else "黑方", "走棋")
            if self.board.is_check():
                print("将军!")
    
    def make_move(self, move_str):
        """执行一步棋"""
        try:
            move = chess.Move.from_uci(move_str)
            if move in self.board.legal_moves:
                self.board.push(move)
                self.display_board()
                
                # 如果有AI模型，让AI走下一步
                if self.model and not self.board.is_game_over():
                    self.make_ai_move()
            else:
                print("非法移动!")
        except ValueError:
            print("移动格式错误! 请使用UCI格式 (例如: e2e4)")
    
    def make_ai_move(self):
        """AI走棋"""
        if self.mcts:
            # 获取当前局面的ChessBoard对象
            chess_board = ChessBoard()
            chess_board.board = self.board.copy()
            
            # 使用MCTS选择最佳移动
            action = self.mcts.get_action(chess_board)
            move = chess.Move.from_uci(action)
            
            # 执行移动
            self.board.push(move)
            self.display_board()
    
    def reset_board(self):
        """重置棋盘"""
        self.board.reset()
        self.display_board()
    
    def undo_move(self):
        """撤销上一步"""
        if len(self.board.move_stack) > 0:
            self.board.pop()
            self.display_board()
        else:
            print("没有可以撤销的移动!")


In [ ]:
# 创建可视化器实例（可以选择性地提供模型路径）
model_path = '../models/latest_model.pth'  # 如果有训练好的模型，请修改这个路径
visualizer = InteractiveGameVisualizer(model_path if os.path.exists(model_path) else None)

# 显示初始棋盘
visualizer.display_board()


In [ ]:
## 使用说明

1. 执行移动：
```python
visualizer.make_move("e2e4")  # 使用UCI格式的移动，例如从e2到e4
```

2. 撤销移动：
```python
visualizer.undo_move()  # 撤销上一步
```

3. 重置棋盘：
```python
visualizer.reset_board()  # 重置到初始状态
```

常见的UCI格式移动示例：
- e2e4: 白方兵前进两格
- e7e5: 黑方兵前进两格
- g1f3: 白方骑士出动
- b8c6: 黑方骑士出动
- f1c4: 白方主教出动
- f8c5: 黑方主教出动
- e1g1: 白方王车易位（短易位）
- e8c8: 黑方王车易位（长易位）
